In [12]:
using Plots
using Roots
using DelimitedFiles
using LinearAlgebra
using NLsolve

In [420]:
#Programa para calcular el  factor de estructura de una mezcla
#de esferas duras cargadas utilizando MSA

#Parámetros globales

global ϕ = 0.1
global z = [ 1, -1]                 #Carga en cada especie en unidades de e
global zd = z[1]/z[2]
global σ = [1, 10]                   #Radio en cada especie, reducido con la especie 1
global σd3=(σ[2]/σ[1])^3
global ρ1=6*ϕ/(π*(σ[1]^3)*(1- zd*σd3))
global ρ = [ρ1,-ρ1*zd]               #Desnsidad reducida de cada especie
global m = 2                         #Número de especies
global σ12=(σ[1] + σ[2])/2
global z12=abs(z[1]*z[2])
#global ϵ = 81.1                      #Constante dieléctrica del agua
global Tred = 0.0307               #Temperatura
global βred = σ12/(z12*Tred)         #Beta reducida


179.1530944625407

In [421]:
#Cálculo de la función de correlación directa en el espacio de fourier
#para mezclas de esferas duras

function Qk(k,i,j)
    
    ζ2 = 0   
    for q=1:m
    ζ2 = ζ2 + ρ[q]*(σ[q]^2)*π/6
    end 
    
    ζ3 = 0   
    for q=1:m
    ζ3 = ζ3 + ρ[q]*(σ[q]^3)*π/6
    end 
    
   
   Sr =(σ[i] - σ[j])/2 
   R = (σ[i] + σ[j])/2
   ai = (1- ζ3 + 3*σ[i]*ζ2)/((1 - ζ3)^2)
   bi = -(3/2)*(σ[i]^2)*ζ2/((1-ζ3)^2)
    
   if i==j
    delta=1 
   else
    delta=0
   end 
 
  
    pR=(ai/2)*exp(im*k*R)*((2-2*im*k*R-(k^2)*(R^2))/(im*k)^3) -(ai/2)*(R^2)*(exp(im*k*R)/(im*k)) +bi*exp(im*k*R)*((im*k*R - 1)/((im*k)^2)) - bi*R*(exp(im*k*R)/(im*k))
    
    pS=(ai/2)*exp(im*k*Sr)*((2-2*im*k*Sr-(k^2)*(Sr^2))/(im*k)^3) -(ai/2)*(R^2)*(exp(im*k*Sr)/(im*k)) +bi*exp(im*k*Sr)*((im*k*Sr - 1)/((im*k)^2)) - bi*R*(exp(im*k*Sr)/(im*k))
    
    Q= delta - 2*π*sqrt(ρ[i]*ρ[j])*(pR-pS)
    
    return Q
end

function Q(k)
        
    Q = [[Qk(k,1,1),Qk(k,2,1)]  [Qk(k,1,2),Qk(k,2,2)] ]

   
    return Q
end

function QT(k)
        
    QT=transpose(Q(-k))
   
    return QT
end

function Shs(k)  
    
    
    S=inv(QT(k)*Q(k))
    
    return real(S)
end

#Función de correlción directa de mezclas de esferas duras

function ρChs(k)  
    
    I= [[1,0] [0,1]]
    
    C=(Shs(k)- I)/(Shs(k))
    
    return C
end

ρChs (generic function with 1 method)

In [422]:
#.................................................................................................
#Función para Xi y Ni necesarias para los cálculos de Cij y cálculo de Γ

cs=0
for j=1:m
     cs= cs + ρ[j]*σ[j]^3
end

global p=(π/2)/(1 - (π/6)*cs)


function X(i,Γ)
               
    X2=0    
    for j=1:m
        X2= X2 + ρ[j]*σ[j]*z[j]/(1 + Γ*σ[j])
    end
    
    X3=0
    for j=1:m
        X3= X3 + ρ[j]*(σ[j]^3)/(1 + Γ*σ[j])
    end
    
    X4= (z[i]/(1+ Γ*σ[i])) - (p*(σ[i]^2)/(1+ Γ*σ[i]))*(X2/(1 + p*X3))
        
    return X4
end

function F(Γ)
    
D=0
for j=1:m
    D= D + ρ[j]*(X(j,Γ)^2)
end   
    
F =  (Γ^2) - π*βred*D

    return F
end

global Γg=find_zero(F, 1)

function N(i)
    
  N=(X(i,Γg) - z[i])/σ[i]

    return N
end


#................................................................................................
#Calculo de la función de correlación directa de mezclas de cargas

function ckelec(k,i,j)
    
    σij=(σ[i] + σ[j])/2
    σmin= (σ[j] - σ[i])/2 
   

#Constante de 0 a  σmin
c1 = -2*(-z[i]*N(j) + X(i,Γg)*(N(i) + Γg*X(i,Γg)) - (σ[i]/3)*((N(i) + Γg*X(i,Γg))^2))    

#Constantes de  σmin a σij
c2 = (σ[i]-σ[j])*(((X(i,Γg) + X(j,Γg))/4)*(N(i) + Γg*(X(i,Γg)-X(j,Γg)) - N(j)) - ((σ[i]-σ[j])/16)*((N(i) + Γg*(X(i,Γg)+X(j,Γg)) + N(j))^2 - 4*N(i)*N(j))) 
c3 = -((X(i,Γg)-X(j,Γg))*(N(i)-N(j)) + (X(i,Γg)^2 + X(j,Γg)^2)*Γg + (σ[i] + σ[j])*N(i)*N(j) -(1/3)*(σ[i]*(N(i) + Γg*X(i,Γg))^2 + σ[j]*(N(j) + Γg*X(j,Γg))^2))
c4 = ((X(i,Γg)/σ[i])*(N(i) + Γg*X(i,Γg)) + (X(j,Γg)/σ[j])*(N(j) + Γg*X(j,Γg)) + N(i)*N(j) -(1/2)*((N(i) + Γg*X(i,Γg))^2 + (N(j) + Γg*X(j,Γg))^2))
c5 = ((1/(6*σ[i]^2))*(N(i) + Γg*X(i,Γg))^2 + (1/(6*σ[j]^2))*(N(j) + Γg*X(j,Γg))^2) 
            
#Constante de  σij a infinito         
c6= -z[i]*z[j]        
                    
I = (sin(k*σmin)-k*σmin*cos(k*σmin))/(k^2)
    
II = (cos(k*σmin) - cos(k*σij))/k
    
III = (sin(k*σij)-k*σij*cos(k*σij) - sin(k*σmin) +k*σmin*cos(k*σmin))/(k^2)
    
IV = ((2 - (k^2)*(σij^2))*cos(k*σij) + 2*k*σij*sin(k*σij) -(2 - (k^2)*(σmin^2))*cos(k*σmin) - 2*k*σmin*sin(k*σmin))/(k^3)
    
V = (4*k*σij*((k^2)*(σij^2)-6)*sin(k*σij) - ((k^4)*(σij^4) - 12*(k^2)*(σij^2) + 24)*cos(k*σij) - 4*k*σmin*((k^2)*(σmin^2)-6)*sin(k*σmin) + ((k^4)*(σmin^4) - 12*(k^2)*(σmin^2) + 24)*cos(k*σmin))/(k^5)
    
VI = cos(k*σij)/k
 
    
c = sqrt(ρ[i]*ρ[j])*(4*π*βred/k)*(c1*I + c2*II + c3*III + c4*IV + c5*V + c6*VI)

    
    return c
end
  
#Funcion de correlación directa de mezcla de cargas
function ρCelec(k)
        
    B = [[ckelec(k,1,1),ckelec(k,2,1) ] [ckelec(k,1,2),ckelec(k,2,2) ] ]

   
    return B
end

ρCelec (generic function with 1 method)

In [423]:
#Matrix de la función de correlación directa de mezclas de esferas duras cargadas
function ρC(k)
        
    B = ρChs(k) + ρCelec(k)
   
    return B
end

#Factor de estructura directa de mezclas de esferas duras cargadas
function S(k)
        
    I= [[1,0] [0,1]]
        
    A = inv(I - ρC(k))
   
    return A
end



S (generic function with 1 method)

In [424]:
#Plots.plot(k ->  S(k)[1,1], 0,40)
#Plots.plot!(k ->  S(k)[2,1], 0,40)
#Plots.plot!(k ->  S(k)[2,2], 0,40)

In [425]:
#Generamos vector de S(k)
global kv = [ 0.05*i for i=1:800]

ST=[S(kv[i]) for i=1:800 ]

S11 = [ST[i][1,1] for i=1:800 ]
S21 = [ST[i][2,1] for i=1:800 ]
S12 = [ST[i][1,2] for i=1:800 ]
S22 = [ST[i][2,2] for i=1:800 ]


function save_data(nombre,formato)
    @assert typeof(nombre) == typeof("hola")
    open(nombre, "w") do io
        writedlm(io,formato)
    end
    println("Data saved as", nombre)
end


SK= save_data("SK04.dat", [kv S11 S12 S21 S22])


Data saved asSK04.dat


In [426]:
#Cálculo del arresto dinámico de una mezcla de esferas duras cargadas

#Matriz de la función de correlacion directa

function C(k)
    
A = [[ρC(k)[1,1]/ρ[1],ρC(k)[2,1]/sqrt(ρ[2]*ρ[1])] [ρC(k)[1,2]/sqrt(ρ[1]*ρ[2]),ρC(k)[2,2]/ρ[2]]]

return A
end

CT=[C(kv[i]) for i=1:800 ]

#Matriz de la función de correlacion total 

function H(k)

A = [[(S(k)[1,1]-1)/ρ[1],(S(k)[2,1])/sqrt(ρ[2]*ρ[1])] [(S(k)[1,2])/sqrt(ρ[1]*ρ[2]),(S(k)[2,2]-1)/ρ[2]]] 

return A
end

HT=[H(kv[i]) for i=1:800 ]

function λ(k)

 kmin=2*π*1.305/σ[1]
 lam=1/(1+(k/kmin)^2)

 λ= [[lam,0] [0,lam]]

return  λ
end

λ (generic function with 1 method)

In [427]:
#Guardamos en archivos HT y CT

C11 = [CT[i][1,1] for i=1:800 ]
C21 = [CT[i][2,1] for i=1:800 ]
C12 = [CT[i][1,2] for i=1:800 ]
C22 = [CT[i][2,2] for i=1:800 ]


CK= save_data("CK.dat", [kv C11 C21 C12 C22])
    
H11 = [HT[i][1,1] for i=1:800 ]
H21 = [HT[i][2,1] for i=1:800 ]
H12 = [HT[i][1,2] for i=1:800 ]
H22 = [HT[i][2,2] for i=1:800 ]

HK= save_data("HK.dat", [kv H11 H21 H12 H22])

Data saved asCK.dat
Data saved asHK.dat


In [428]:
#Las siguientes funciones son para utilizar el metodo de punto fijo

function γ(γ1,γ2)
    
    #Matriz de raiz de densidades
    n2= [[sqrt(ρ[1]),0] [0,sqrt(ρ[2])]]
    
    #Matriz de gamma
    γ=[[γ1,0] [0,γ2]]
    
    I= [[1,0] [0,1]]
          
    dk = kv[2]-kv[1]
    suma1 = 0.0
    suma2 = 0.0
    i=0
    
    for i=1:800-1
        
        i=i+1
        
        A=inv(I + (kv[i]^2)*γ*inv(λ(kv[i])))
        B=CT[i]*n2*inv(I + (kv[i]^2)*γ*inv(λ(kv[i]))*inv(ST[i]))*n2*HT[i]

         suma1 = suma1 + (kv[i]^4)*A[1,1]*B[1,1]
        
         suma2 = suma2 + (kv[i]^4)*A[2,2]*B[2,2]

    end
          int1=suma1*dk/(6*pi^2)
          int2=suma2*dk/(6*pi^2)
 
    γ11= 1/int1
    γ22= 1/int2


    
    return γ11, γ22
end


γ (generic function with 1 method)

In [429]:
inf=1.0e8 
tol=1.0e-7
errorγ1= 1.0
errorγ2= 1.0
γ1test=1.0e-6
γ2test=1.0e-6
i=0

     while  tol<errorγ2 # ||  tol<errorγ2
     
           i=i+1       
    
           γp=γ(γ1test,γ2test)
           γ1=γp[1]
           γ2=γp[2]
    
           errorγ1=abs(1- γ1/γ1test)
           errorγ2=abs(1- γ2/γ2test)
    
           #println(i," "," ",Float64(γ2))
    
           γ1test=γ1
           γ2test=γ2 
    
     #if (inf<γ1 ||  inf<γ2) break
    
    if (inf<γ2) break
    
    end
    
    end

In [430]:
γ2test

23.94754601383124